In [44]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
)

examples = [
    {
        "question": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
        
    },
    {
        "question": "Thor",
        "answer": "⚡️🔨🌩️",
    },
    {
        "question": "Spider Man",
        "answer": "🕷️🕸️🗽",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{question}"), ("ai", "{answer}")]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert. You know every movie. If a human tells you the title of the movie, you have to respond with 3 emoticons."),
        few_shot_prompt,
        (
            "system",
            "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"), 
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [31]:
print(load_memory(_))

[]


In [32]:
invoke_chain("Squid Game")

content='🦑💰🎮'


In [40]:
invoke_chain("Captain America")

content='🇺🇸⭐🛡️'


In [41]:
print(load_memory(_))

[HumanMessage(content='What were the movies I asked before?'), AIMessage(content='You asked about "Top Gun," "The Godfather," "Thor," and "Spider Man."'), HumanMessage(content='Captain America'), AIMessage(content='🇺🇸⭐🛡️')]


In [ ]:
invoke_chain("What were all the movies I asked before?")

content='Before "Top Gun," you asked about "The Godfather."'


In [46]:
invoke_chain("What was the first movie I asked?")

content='You asked about "Top Gun."'


In [19]:
print(load_memory(_))

[HumanMessage(content='Squid Game'), AIMessage(content='🦑🎮💰'), HumanMessage(content='old boy'), AIMessage(content='👴👦🔨')]


In [45]:
memory.clear()